# LDA MODEL USING GENSIM LIBRARY 
Starting from the initial dataset, preprocess it and then apply the LDA model of gensim library. 
Finally, having the output, perform a binary classification analysis

## import libraries

In [ ]:
!pip install pyLDAvis &> /dev/null

In [ ]:
import numpy as np
import pandas as pd

import json
import glob

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [ ]:
from gensim.models import LdaModel, LdaMulticore
import gensim.downloader as api
from gensim.utils import simple_preprocess, lemmatize
from nltk.corpus import stopwords
import re
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)

In [ ]:
import pickle
from pprint import pprint
import collections
import numpy 
import matplotlib.pyplot as plt
import operator
import array
import pandas as pd


# Natural Language Tool Kit (NLTK) imports
import nltk
from nltk.data  import load
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

# Machine Learning Library (sklearn) imports
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer , CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer

from gensim.models import TfidfModel

In [ ]:
# Install nltk, Library suited for text mining and texts processing
import nltk
nltk.download('punkt') # containing the tokeinzers
import nltk
nltk.download('stopwords')  # Frequent or meaningless words of,an,a...

# Split a text into sentences
sentence_splitter = load('tokenizers/punkt/english.pickle')

# Split a sentence into words
tokenizer = TreebankWordTokenizer()

# Reduce to word to the root
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
punctuation = set([",", ".", ";", "/", ":", "-", "--" ,"!", "?", "(", ")","'",'"',"''", "``"])

# Set of common english words
stopwords_set = set(stopwords.words('english'))  # very common words in english
 

## text processing functions

In [ ]:
# Preprocess a given text
def preprocess_text(review):
    tokens = []
    # A. Split into sentences
    for sentence in sentence_splitter.tokenize(review):
        # B. Split into tokens
        for token in tokenizer.tokenize(sentence):
            token = token.lower()
            # C. Filter on stoplist and punctuation
            if token not in stopwords_set and token not in punctuation:
                # D. Stemming (takes root)
                stem = stemmer.stem(token)
                tokens.append(stem)
    return tokens

## Import of datasets

Dataset sul drive condiviso: positive_text.review e negative_text.review

Importo le recensioni per ognuno dei 4 argomenti

In [ ]:
#RUN THIS CELL ONLY TO IMPORT THE BOOKS REVIEWS
# Import the set of positive book reviews
with open('/content/positive_text_book.review', 'r') as myfile: #positive reviwes about books
    pos_books = myfile.readlines()

# Import the set of negative book reviews
with open('/content/negative_text_book.review', 'r') as myfile: #negative reviwes about books
    neg_books = myfile.readlines()

In [ ]:
# Import the set of positive book reviews
with open('/content/positive_text_book.review', 'r') as myfile: #positive reviwes about books
    pos_books = myfile.readlines()

# Import the set of negative book reviews
with open('/content/negative_text_book.review', 'r') as myfile: #negative reviwes about books
    neg_books = myfile.readlines()

# Import the set of positive electronic reviews
with open('/content/positive_text_elec.review', 'r') as myfile: #positive reviwes about books
    pos_elec = myfile.readlines()

# Import the set of negative electronic reviews
with open('/content/negative_text_elec.review', 'r') as myfile: #negative reviwes about books
    neg_elec = myfile.readlines()

# Import the set of positive kitchen reviews
with open('/content/positive_text_kitch.review', 'r') as myfile: #positive reviwes about books
    pos_kitch = myfile.readlines()

# Import the set of negative kitchen reviews
with open('/content/negative_text_kitch.review', 'r') as myfile: #negative reviwes about books
    neg_kitch = myfile.readlines()

# Import the set of positive dvd reviews
with open('/content/positive_text_dvd.review', 'r') as myfile: #positive reviwes about books
    pos_dvd = myfile.readlines()

# Import the set of negative dvd reviews
with open('/content/negative_text_dvd.review', 'r') as myfile: #negative reviwes about books
    neg_dvd = myfile.readlines()


In [ ]:
print("pos_books length : " , len(pos_books) , "\n")
print("neg_books length : " , len(neg_books))

pos_books length :  1000 

neg_books length :  1000


In [ ]:
print(pos_books[0])

 Sphere by Michael Crichton is an excellant novel. This was certainly the hardest to put down of all of the Crichton novels that I have read.   The story revolves around a man named Norman Johnson. Johnson is a phycologist. He travels with 4 other civilans to a remote location in the Pacific Ocean to help the Navy in a top secret misssion. They quickly learn that under the ocean is a half mile long spaceship. The civilans travel to a center 1000 feet under the ocean to live while researching the spacecraft. They are joined by 5 Navy personel to help them run operations. However on the surface a typhoon comes and the support ships on the surface must leave. The team of ten is stuck 1000 feet under the surface of the ocean. After a day under the sea they find out that the spacecraft is actually an American ship that has explored black holes and has brought back some strange things back to earth.  This novel does not have the research that some of the other Crichton novels have, but it st

## Preprocessing of the dataset

Creo un dataset di recensioni lungo 8000 (2000 per ogni argomento)

In [ ]:
#BOOKS REVIEWS

# Dictionary of texts
B_pos={}
B_neg={}

# All possible words
features=[]



for i in range(1000):
  # Preprocess the i-th text
  pre_pos=preprocess_text(pos_books[i].replace("'",' '))

  pre_neg=preprocess_text(neg_books[i].replace("'",' '))

  # Add the text to the dictionary
  B_pos[i]=pre_pos

  B_neg[i]=pre_neg

  # Add all the tokens to the list
  for j in pre_pos:
    features.append(j)

  for j in pre_neg:
    features.append(j)

    
    

# list to set 
features=set(features)


# set to dictionary
feat={}

k=0
# Each word in features is associated to an integer (index of columns)
for i in features:
   feat[i]=k
   k+=1


In [ ]:
a = list(B_pos.values())
b = list(B_neg.values())

data = a + b
len(data)

2000

In [ ]:
#ELECTRONIC REVIEWS

# Dictionary of texts
B_pos={}
B_neg={}

# All possible words
features=[]



for i in range(1000):
  # Preprocess the i-th text
  pre_pos=preprocess_text(pos_elec[i].replace("'",' '))

  pre_neg=preprocess_text(neg_elec[i].replace("'",' '))

  # Add the text to the dictionary
  B_pos[i]=pre_pos

  B_neg[i]=pre_neg

  # Add all the tokens to the list
  for j in pre_pos:
    features.append(j)

  for j in pre_neg:
    features.append(j)

    
    

# list to set 
features=set(features)


# set to dictionary
feat={}

k=0
# Each word in features is associated to an integer (index of columns)
for i in features:
   feat[i]=k
   k+=1

In [ ]:
a = list(B_pos.values())
b = list(B_neg.values())

data = data + a + b
len(data)

4000

In [ ]:
# KITCHEN REVIEWS

# Dictionary of texts
B_pos={}
B_neg={}

# All possible words
features=[]



for i in range(1000):
  # Preprocess the i-th text
  pre_pos=preprocess_text(pos_kitch[i].replace("'",' '))

  pre_neg=preprocess_text(neg_kitch[i].replace("'",' '))

  # Add the text to the dictionary
  B_pos[i]=pre_pos

  B_neg[i]=pre_neg

  # Add all the tokens to the list
  for j in pre_pos:
    features.append(j)

  for j in pre_neg:
    features.append(j)

    
    

# list to set 
features=set(features)


# set to dictionary
feat={}

k=0
# Each word in features is associated to an integer (index of columns)
for i in features:
   feat[i]=k
   k+=1

In [ ]:
a = list(B_pos.values())
b = list(B_neg.values())

data = data + a + b
len(data)

6000

In [ ]:
# DVD REVIEWS

# Dictionary of texts
B_pos={}
B_neg={}

# All possible words
features=[]



for i in range(1000):
  # Preprocess the i-th text
  pre_pos=preprocess_text(pos_dvd[i].replace("'",' '))

  pre_neg=preprocess_text(neg_dvd[i].replace("'",' '))

  # Add the text to the dictionary
  B_pos[i]=pre_pos

  B_neg[i]=pre_neg

  # Add all the tokens to the list
  for j in pre_pos:
    features.append(j)

  for j in pre_neg:
    features.append(j)

    
    

# list to set 
features=set(features)


# set to dictionary
feat={}

k=0
# Each word in features is associated to an integer (index of columns)
for i in features:
   feat[i]=k
   k+=1

In [ ]:
a = list(B_pos.values())
b = list(B_neg.values())

data = data + a + b
len(data)

8000

##LDA model with gensim: base one

In [ ]:
import gensim

# Create the bigram model using the Phrases class
bigram_model = gensim.models.Phrases(data, min_count=7, threshold=20)

# Create a Phraser object from the bigram model
bigram_phraser = gensim.models.phrases.Phraser(bigram_model)

# Apply the bigram model to the tokens
bigrams = [bigram_phraser[token] for token in data]

# Create the trigram model using the Phrases class
#trigram_model = gensim.models.Phrases(bigrams, min_count=7, threshold=20)

# Create a Phraser object from the trigram model
#trigram_phraser = gensim.models.phrases.Phraser(trigram_model)

# Apply the trigram model to the bigrams
#trigrams = [trigram_phraser[bigram] for bigram in bigrams]

INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO:gensim.models.phrases:collected 423503 word types from a corpus of 553251 words (unigram + bigrams) and 8000 sentences
INFO:gensim.models.phrases:using 423503 counts as vocab in Phrases<0 vocab, min_count=7, threshold=20, max_vocab_size=40000000>
INFO:gensim.models.phrases:source_vocab length 423503
INFO:gensim.models.phrases:Phraser built with 716 phrasegrams


Rimuovo parole che secondo me non hanno senso per classificare positivo o negativo

In [ ]:
list_proibita = ['kagan', 'christian', 'john', 'japanes', 'lawrenc', 'jimi', 'x', '*', '[', ']', 'book', 'read']

In [ ]:
super = [[word for word in doc if word.lower() not in list_proibita] for doc in bigrams]

In [ ]:
dct = corpora.Dictionary(super)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(33371 unique tokens: ['1000', '4', '5', 'actual', 'american']...) from 8000 documents (total 530009 corpus positions)


In [ ]:
corpus = [dct.doc2bow(line) for line in super]

Devo rimuovere le parole che appaiono molto spesso nel dataset

In [ ]:
tfidf = TfidfModel(corpus, id2word=dct)

low_value = 0.1 #tune this parameters 
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
  bow = corpus[i]
  low_value_words = []
  tfidf_ids = [id for id, value in tfidf[bow]]
  bow_ids = [id for id, value in bow]
  low_value_words = [id for id, value in tfidf[bow] if value < low_value]
  drops = low_value_words+words_missing_in_tfidf
  for item in drops:
    words.append(dct[item])
  words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

  new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
  corpus[i] = new_bow

INFO:gensim.models.tfidfmodel:collecting document frequencies
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #0
INFO:gensim.models.tfidfmodel:calculating IDF weights for 8000 documents and 33370 features (423751 matrix non-zeros)


In [ ]:
num_topics = 4

In [ ]:
lda_model = LdaMulticore(corpus=corpus,
                         id2word=dct,
                         random_state=100,
                         num_topics= num_topics,
                         passes= 10,
                         chunksize= 1000,
                         #batch=False,
                         #alpha= 'symmetric',
                         decay=0.7,
                         offset= 1,
                         eta= 'symmetric',
                         per_word_topics=True)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.25
INFO:gensim.models.ldamodel:using symmetric eta at 0.25
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamulticore:running online LDA training, 4 topics, 10 passes over the supplied corpus of 8000 documents, updating every 1000 documents, evaluating every ~8000 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamulticore:training LDA model using 1 processes
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #0 = documents up to #1000/8000, outstanding queue size 1
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #1 = documents up to #2000/8000, outstanding queue size 2
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #2 = documents up to #3000/8000, outstanding queue size 3
INFO:gensim.models.ldamodel:merging changes from 1000 documents into a model of 8000 documents
INFO:gensim.models.ldamodel:topic #0

In [ ]:
#visualize LDA
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, corpus, dct, mds = 'mmds', R = 30)

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.230172  0.083107       1        1  41.029424
2      0.071751 -0.157223       2        1  28.849359
1     -0.199083 -0.091574       3        1  15.570733
0     -0.102840  0.165690       4        1  14.550484, topic_info=         Term         Freq        Total Category  logprob  loglift
1599     film  1656.000000  1656.000000  Default  30.0000  30.0000
717      movi  2393.000000  2393.000000  Default  29.0000  29.0000
9231      dvd   681.000000   681.000000  Default  28.0000  28.0000
424     watch   451.000000   451.000000  Default  27.0000  27.0000
673   product   533.000000   533.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
541     child    44.662648    87.029050   Topic4  -6.6386   1.2604
125       way    48.936639   132.717606   Topic4  -6.5472   0.9298
1942     cook    55.036335   334.150196   Topic4  -6.4298   0.1239
717      movi    61.749000  2393.726268   Topic4  -6.3147  -1.7300
412    famili    43.749462   166.946993   Topic4  -6.6593   0.5883

[266 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2231      1  0.969992        $
2231      4  0.024557        $
7534      4  0.956449       ..
533       1  0.272832      ...
533       2  0.530039      ...
...     ...       ...      ...
182       4  0.008007    worst
700       1  0.040788  written
700       2  0.938114  written
700       4  0.020394  written
127       3  0.981045     yoga

[461 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1])

## Evaluation of the base model: coherence score

In [ ]:
# Compute Coherence Score of the base model
coherence_model_lda = CoherenceModel(model=lda_model, texts= super, dictionary= dct, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

INFO:gensim.topic_coherence.probability_estimation:using WordOccurrenceAccumulator to estimate probabilities from sliding windows
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 3000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 4000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 5000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 6000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 7000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 8000 documents
INFO:gensim.topic_cohe


Coherence Score:  0.4067871526135456


## LDA MALLET model

In [ ]:
!pip install --upgrade gensim==3.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.4 MB 1.3 MB/s 
  Created wheel for gensim: filename=gensim-3.8.0-cp38-cp38-linux_x86_64.whl size=25868892 sha256=0a8a795f2e5371f05152eb081c445d215d03453b4d2d2797def9d4108d74d9dc
  Stored in directory: /root/.cache/pip/wheels/09/d1/44/6d6dcb820e282b40ccfce6d2bb0101c24d7fffd863b2276783
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


In [ ]:
!unzip mallet-2.0.8.zip

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
  inflating: mallet-2.0.8/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/mallet-2.0.8/
  inflating: __MACOSX/mallet-2.0.8/._.DS_Store  
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
   creating: __MACOSX/mallet-2.0.8/bin/
  inflating: __MACOSX/mallet-2.0.8/bin/._classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: __MACOSX/mallet-2.0.8/bin/._csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: __MACOSX/mallet-2.0.8/bin/._csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: __MACOSX/mallet-2.0.8/bin/._mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: __MACOSX/mallet-2.0.8/bin/._mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: __MACOSX/mallet-2.0.8/bin/._mallethon  
  inflating: mallet-2.0.8/bin/prepend-license.sh  
  inflating: __MACOSX/mallet-2.0.8/bin/._prepend-license.sh  
  inflating: mallet-2.0.8/bi

In [ ]:
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet'

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=8, id2word=dct)

INFO:gensim.models.wrappers.ldamallet:serializing temporary corpus to /tmp/716170_corpus.txt
INFO:gensim.models.wrappers.ldamallet:converting temporary corpus to MALLET format with /content/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/716170_corpus.txt --output /tmp/716170_corpus.mallet
INFO:gensim.models.wrappers.ldamallet:training MALLET LDA with /content/mallet-2.0.8/bin/mallet train-topics --input /tmp/716170_corpus.mallet --num-topics 8  --alpha 50 --optimize-interval 0 --num-threads 4 --output-state /tmp/716170_state.mallet.gz --output-doc-topics /tmp/716170_doctopics.txt --output-topic-keys /tmp/716170_topickeys.txt --num-iterations 1000 --inferencer-filename /tmp/716170_inferencer.mallet --doc-topics-threshold 0.0
INFO:gensim.models.wrappers.ldamallet:loading assigned topics from /tmp/716170_state.mallet.gz


In [ ]:
# Compute Coherence Score of the base model
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts= super, dictionary= dct, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

INFO:gensim.topic_coherence.probability_estimation:using WordOccurrenceAccumulator to estimate probabilities from sliding windows
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 3000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 4000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 5000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 6000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 7000 documents
INFO:gensim.topic_coherence.text_analysis:WordOccurrenceAccumulator accumulated stats from 8000 documents
INFO:gensim.topic_cohe


Coherence Score:  0.3724672230779549


## Post LDA analysis: CLASSIFICATION

In [ ]:
#for the mallet model run this
model_mallet = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
topic_distribution = model_mallet.get_document_topics(corpus, minimum_probability=0)

INFO:gensim.models.ldamodel:using symmetric eta at 0.125
INFO:gensim.models.ldamodel:using serial LDA version on this node


In [ ]:
#for the normal model run this
topic_distribution = lda_model.get_document_topics(corpus, minimum_probability=0)

In [ ]:
num_documents = len(corpus)
num_documents

8000

In [ ]:
new_data = np.zeros((num_documents, num_topics)) 

for i in range(new_data.shape[0]):
  topics = [id for id,value in topic_distribution[i]]
  values = [value for id,value in topic_distribution[i]]
  for z in range(len(values)):
    new_data[i,topics[z]] = values[z]

new_data.shape

(8000, 4)

In [ ]:
data_mean = new_data.mean()
data_std = new_data.std()
data_normalized = (new_data - data_mean) / data_std

In [ ]:
import numpy as np
# Create two arrays of 1000 elements each, one with all ones and one with all zeros
ones = np.ones(1000)
zeros = np.zeros(1000)

# Concatenate the two arrays using the concatenate function
y = np.concatenate((ones, zeros, ones, zeros, ones, zeros, ones, zeros))
#y = y[:2000]
y.shape

(8000,)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_normalized, y, test_size=0.25)

Insert here the model:

It can be a model that classify between positive/negative reviews or by the 4 topics